In [1]:
import dotenv
import os
from langchain_community.utilities.google_search import GoogleSearchAPIWrapper
from langchain_community.chat_models.openai import ChatOpenAI
from discussion_agents.cog.functional.critic import (
    _prompt_agent,
    _prompt_critique,
    _build_critique_format_prompt,
)
from discussion_agents.cog.prompts.critic import (
    HOTPOTQA_FEWSHOT_EXAMPLES_COT,
    HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC,
    CRITIC_CRITIQUE_FORMAT_HOTPOTQA,
)

dotenv.load_dotenv("../../../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")
google_cse_id = os.getenv("GOOGLE_CSE_ID")
google_api_key = os.getenv("GOOGLE_API_KEY")
llm = ChatOpenAI(openai_api_key=openai_api_key)

In [2]:
question = 'Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring'

answer = _prompt_agent(
    llm=llm, question=question, examples=HOTPOTQA_FEWSHOT_EXAMPLES_COT
)

In [3]:
answer

"Let's break it down step by step. The individual in question was once considered the best kickboxer in the world but has been involved in controversies related to unsportsmanlike conduct and crimes of violence. Based on the description, the individual being referred to is Badr Hari, a Moroccan-Dutch kickboxer known for his aggressive style and controversial behavior both inside and outside the ring. So the answer is: Badr Hari."

In [4]:
critique = _prompt_critique(
    llm=llm, question=question, examples=HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC, answer=answer
).split("> Evidence: ")[0]

In [5]:
print(critique)

The question asks for the name of an individual fitting the description provided, and the answer "Badr Hari" is a plausible response.

2. Truthfulness:

Let's search the question in Google:

> Search Query: Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring



In [6]:
from discussion_agents.cog.prompts.critic import CRITIC_CRITIQUE_INSTRUCTION_HOTPOTQA

In [7]:
a = CRITIC_CRITIQUE_INSTRUCTION_HOTPOTQA.format(
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC, question=question, answer=answer
)

In [8]:
print(a + critique)

Question: Serianna is a band of what genre that combines elements of heavy metal and hardcore punk?
Proposed Answer: Let's think step by step. Serianna is a band of metalcore genre. Metalcore is a subgenre of heavy metal and hardcore punk. So Serianna is a band of heavy metal and hardcore punk. So the answer is: heavy metal and hardcore punk.

What's the problem with the above answer?

1. Plausibility:

The question asks for the genre that combines elements of heavy metal and hardcore punk, and the answer is "heavy metal and hardcore punk", simply repeat the question. So it's not plausible.

2. Truthfulness:

Let's search the question in google:
> Search Query: Serianna is a band of what genre that combines elements of heavy metal and hardcore punk? site: wikipedia.org
> Evidence: [Metalcore - Wikipedia] Metalcore is a fusion music genre that combines elements of extreme metal and hardcore punk.

The evidence suggests that metalcore is a genre that combines elements of extreme metal an

In [9]:
_, search_query = critique.split("> Search Query:")[:2]
search_query = search_query.split("\n")[0].strip()

In [10]:
formatted_critique = _build_critique_format_prompt(
    question=question,
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC,
    answer=answer,
    critique=critique,
    prompt=CRITIC_CRITIQUE_FORMAT_HOTPOTQA,
)

In [11]:
formatted_critique == a + critique

True

In [12]:
print(formatted_critique)

Question: Serianna is a band of what genre that combines elements of heavy metal and hardcore punk?
Proposed Answer: Let's think step by step. Serianna is a band of metalcore genre. Metalcore is a subgenre of heavy metal and hardcore punk. So Serianna is a band of heavy metal and hardcore punk. So the answer is: heavy metal and hardcore punk.

What's the problem with the above answer?

1. Plausibility:

The question asks for the genre that combines elements of heavy metal and hardcore punk, and the answer is "heavy metal and hardcore punk", simply repeat the question. So it's not plausible.

2. Truthfulness:

Let's search the question in google:
> Search Query: Serianna is a band of what genre that combines elements of heavy metal and hardcore punk? site: wikipedia.org
> Evidence: [Metalcore - Wikipedia] Metalcore is a fusion music genre that combines elements of extreme metal and hardcore punk.

The evidence suggests that metalcore is a genre that combines elements of extreme metal an

In [13]:
exist_query = []
exist_evidence = set()
search = GoogleSearchAPIWrapper(
    google_cse_id=google_cse_id, google_api_key=google_api_key
)

exist_query.append(search_query)
for k in range(exist_query.count(search_query), 8):
    search_result = search.results(search_query, num_results=k)[-1]
    if search_result["snippet"] not in exist_evidence:
        exist_evidence.add(search_result["snippet"])
        break

In [14]:
exist_query

['Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring']

In [15]:
print(exist_evidence)

{'Who was once considered the best kick boxer in the world, however he has ... unsportsmanlike conducts" in the sport and crimes of violence outside of the ring.'}


In [16]:
print(search_result)

{'title': 'datadreamer-dev/hotpot_qa_augmented · Datasets at Hugging Face', 'link': 'https://huggingface.co/datasets/datadreamer-dev/hotpot_qa_augmented', 'snippet': 'Who was once considered the best kick boxer in the world, however he has ... unsportsmanlike conducts" in the sport and crimes of violence outside of the ring.'}


In [17]:
context = (
    f"""> Evidence: [{search_result['title']}] {search_result['snippet'][:400]}\n\n"""
)
context += f"Let's give the most possible answer.\n\nQuestion: {question}\nHere's "

In [18]:
print(context)

> Evidence: [datadreamer-dev/hotpot_qa_augmented · Datasets at Hugging Face] Who was once considered the best kick boxer in the world, however he has ... unsportsmanlike conducts" in the sport and crimes of violence outside of the ring.

Let's give the most possible answer.

Question: Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring
Here's 


In [19]:
formatted_critique += context

In [20]:
print(formatted_critique)

Question: Serianna is a band of what genre that combines elements of heavy metal and hardcore punk?
Proposed Answer: Let's think step by step. Serianna is a band of metalcore genre. Metalcore is a subgenre of heavy metal and hardcore punk. So Serianna is a band of heavy metal and hardcore punk. So the answer is: heavy metal and hardcore punk.

What's the problem with the above answer?

1. Plausibility:

The question asks for the genre that combines elements of heavy metal and hardcore punk, and the answer is "heavy metal and hardcore punk", simply repeat the question. So it's not plausible.

2. Truthfulness:

Let's search the question in google:
> Search Query: Serianna is a band of what genre that combines elements of heavy metal and hardcore punk? site: wikipedia.org
> Evidence: [Metalcore - Wikipedia] Metalcore is a fusion music genre that combines elements of extreme metal and hardcore punk.

The evidence suggests that metalcore is a genre that combines elements of extreme metal an